In [1]:
import mrcfile
import file
from pathlib import Path
import glob
import numpy as np
import image_process
import matplotlib.pyplot as plt
import cv2

In [2]:
mrc_search_path = '/mnt/experiment/TEM diffraction/'
mrc_file_paths = [str(i) for i in Path(mrc_search_path).rglob("*.mrc")]
random_mrc_files = np.random.choice(mrc_file_paths,10)

In [3]:
mrc_img = mrcfile.open(random_mrc_files[0])
raw_img = mrc_img.data
img = np.array(raw_img)
center = image_process.get_center(img, (120,130),10)

initial center is  (1212.3682117357025, 1081.028529122187)
calculated center is  [1221 1071]


In [4]:
center2 = image_process.get_center_gradient(img, (120,130),10)
center2

initial center is  (1212, 1081)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0

KeyboardInterrupt: 

In [4]:
azavg = image_process.get_azimuthal_average(mrc_img.data,center)[0]
azavg.max()
new_azavg = azavg * np.arange(0,len(azavg))
new_azavg = new_azavg*azavg.max()/new_azavg.max()
new_two = azavg * np.power(np.arange(0,len(azavg)),2)
new_two = new_two *azavg.max()/new_two.max()

In [17]:
img = cv2.normalize(img,img,0,255,cv2.NORM_MINMAX)

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [31]:
rows, cols= 2,2
fig = make_subplots(rows,cols)
for i in range(rows*cols):
    fig.add_trace(
        go.Scatter(x=np.arange(0,len(azavg)),y=azavg, name="default"),
    )
    fig.add_trace(
        go.Scatter(x=np.arange(0,len(azavg)),y=new_azavg, name="deafult*x_axis"),
    )
    fig.add_trace(
        go.Scatter(x=np.arange(0,len(azavg)),y=new_two, name="deafult*x_axis^2"),
    )
    fig.update_layout(
        title_text="azimuthal average",
        hovermode='x',
    #     template="plotly_dark"
    )
fig.show()

In [13]:
fig = make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(
    go.Scatter(x=np.arange(0,len(azavg)),y=azavg, name="default"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=np.arange(0,len(azavg)),y=new_azavg, name="deafult*x_axis"),
    secondary_y=True,
)
fig.update_yaxes(
    title_text="<b>primary</b> yaxis title", 
    secondary_y=False)
fig.update_yaxes(
    title_text="<b>secondary</b> yaxis title", 
    secondary_y=True)
fig.update_layout(hovermode='x')
fig.show()

In [10]:
class Hello:
    print("It happend just once")
    a = 3
    def __init__(self):
        pass

hi = Hello()
hi2 = Hello()

It happend just once


In [14]:
hi2.a

3